In [ ]:
import requests
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import folium
import shap

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


endtime = dt.date.today().isoformat()
starttime = (dt.date.today() - dt.timedelta(days=365*2)).isoformat()

url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
params = {
    "format": "geojson",
    "starttime": starttime,
    "endtime": endtime,
    "minmagnitude": 4.5
}

response = requests.get(url, params=params)
data = response.json()

records = []
for f in data["features"]:
    p = f["properties"]
    c = f["geometry"]["coordinates"]
    records.append({
        "latitude": c[1],
        "longitude": c[0],
        "depth_km": c[2],
        "magnitude": p["mag"]
    })

df = pd.DataFrame(records).dropna()
df["strength_class"] = (df["magnitude"] >= 6.0).astype(int)

print("Total earthquakes:", len(df))


plt.figure(figsize=(8,5))
plt.hist(df["magnitude"], bins=25, edgecolor="black")
plt.xlabel("Magnitude")
plt.ylabel("Frequency")
plt.title("Earthquake Magnitude Distribution")
plt.show()


X = df[["latitude", "longitude", "depth_km"]]
y = df["strength_class"]

model = RandomForestClassifier(
    n_estimators=200,
    class_weight="balanced",
    random_state=42
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

for train, test in cv.split(X, y):
    model.fit(X.iloc[train], y.iloc[train])
    preds = model.predict(X.iloc[test])
    scores.append(accuracy_score(y.iloc[test], preds))

print("CV Accuracy:", np.mean(scores))

model.fit(X, y)


X_sample = X.sample(150, random_state=42)

explainer = shap.TreeExplainer(model)
shap_values = explainer(X_sample)

shap.summary_plot(
    shap_values.values,
    X_sample,
    feature_names=["Latitude", "Longitude", "Depth (km)"]
)


world_map = folium.Map(location=[0, 0], zoom_start=2)

for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=3,
        color="red" if row["strength_class"] == 1 else "blue",
        fill=True,
        fill_opacity=0.6
    ).add_to(world_map)

world_map


plt.figure(figsize=(8,6))
plt.scatter(
    df["magnitude"],
    df["depth_km"],
    c=df["strength_class"],
    alpha=0.6
)
plt.gca().invert_yaxis()
plt.xlabel("Magnitude")
plt.ylabel("Depth (km)")
plt.title("Seismic Depth–Magnitude Profile")
plt.colorbar(label="Strength Class")
plt.show()